In [1]:
import pandas as pd
import urllib

import numpy as np

import json

from tqdm.autonotebook import tqdm

#%matplotlib inline

tqdm.pandas()

import dask.dataframe as dd

from dask.multiprocessing import get
from dask.diagnostics import ProgressBar

from datetime import datetime
import matplotlib.pyplot as plt

from IPython.display import display


/tmp/ipykernel_6191/3514129619.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
import urllib3

In [3]:
http = urllib3.PoolManager()

In [4]:
from config_batch import * 

# Functions

In [5]:
ws_hostname = "127.0.1.1"
ws_hostname = "172.27.0.64"

# ws_hostname = "192.168.1.3"


In [83]:
def call_ws(addr_data, check_result=True, structured_osm=False): #lg = "en,fr,nl"
    t = datetime.now()
    
    params = urllib.parse.urlencode({"street": addr_data[street_field],
                                     "housenumber": addr_data[housenbr_field],
                                     "city": addr_data[city_field],
                                     "postcode": addr_data[postcode_field],
                                     "country": addr_data[country_field],
                                     "check_result" : "yes" if check_result else "no",
                                     "struct_osm" : "yes" if structured_osm else "no"
                                    })
    url = f"http://{ws_hostname}:5000/search/?{params}"
    
#     print(url)
    try:
        with urllib.request.urlopen(url) as response:
            res = response.read()
            res = json.loads(res)
#             print(res)
            res["time"] = (datetime.now() - t).total_seconds()
            return res
    except Exception as e:
        return str(e)
    

In [189]:
def call_ws_test(addr_data, check_result=True, structured_osm=False): #lg = "en,fr,nl"
    t = datetime.now()
    
    params = urllib.parse.urlencode({"street": addr_data[street_field],
                                     "housenumber": addr_data[housenbr_field],
                                     "city": addr_data[city_field],
                                     "postcode": addr_data[postcode_field],
                                     "country": addr_data[country_field],
                                     "check_result" : "yes" if check_result else "no",
                                     "struct_osm" : "yes" if structured_osm else "no"
                                    })
    url = f"http://{ws_hostname}:5000/search_async/?{params}"
    
#     print(url)
    try:
        with urllib.request.urlopen(url) as response:
            res = response.read()
            res = json.loads(res)
#             print(res)
            res["time"] = (datetime.now() - t).total_seconds()
            return res
    except Exception as e:
        return str(e)
    

In [64]:
def call_ws_batch(addr_data, mode="geo", with_reject=False, check_result=True, structured_osm=False): #lg = "en,fr,nl"
#     print(addr_data)
#     print(addr_data.shape)
#     print()
    file_data = addr_data.rename(columns = {
        street_field : "street",
        housenbr_field: "housenumber",
        postcode_field: "postcode",
        city_field: "city",
        country_field: "country",
        addr_key_field : "addr_key"
    }).to_csv(index=False)
    
    r = http.request(
    'POST',
    f'http://{ws_hostname}:5000/batch',
    fields= { 
        'media': ('addresses.csv', file_data),
        'mode': mode,
        "with_rejected" : "yes" if with_reject else "no",
        "check_result" : "yes" if check_result else "no",
        "struct_osm" : "yes" if structured_osm else "no",
        #"extra_house_nbr": "no"
    })
    
    try:
        res = pd.DataFrame(json.loads(r.data.decode('utf-8')))
    except ValueError:
        print("Cannot decode result:")
        print(r.data.decode('utf-8'))
        print(json.loads(r.data.decode('utf-8')))
        return 
#     display(res)
    return res

In [9]:
def expand_json(addresses):
    addresses["status"]= addresses.json.apply(lambda d: "error" if "error" in d else "match" if "match" in d else "rejected")
    addresses["time"]  = addresses.json.apply(lambda d: d["time"])

    addresses["timing"]  = addresses.json.apply(lambda d: d["timing"] if "timing" in d else {})

    addresses["method"]= addresses.json.apply(lambda d: d["match"][0]["method"] if len(d)>0 and "match" in d else "none")
    
    for field in ["street", "number", "postcode", "city"]:
        addresses[field]= addresses.json.apply(lambda d: d["match"][0]["addr_out_"+field] if len(d)>0 and "match" in d else "")
    return 

# Calls

## Single address calls

In [229]:
res=call_ws({street_field:   "Av. Fonsny", \
         housenbr_field: "20",\
         city_field:     "Saint-Gilles",\
         postcode_field: "1060",\
         country_field:  "Belgium"}, check_result=False, structured_osm=False)
res

{'match': [{'SIM_street_which': '',
   'addr_out_city': 'Saint-Gilles - Sint-Gillis',
   'addr_out_country': 'België / Belgique / Belgien',
   'addr_out_number': '20',
   'addr_out_postcode': '1060',
   'addr_out_street': 'Avenue Fonsny - Fonsnylaan',
   'display_name': '20, Avenue Fonsny - Fonsnylaan, Saint-Gilles - Sint-Gillis, Brussel-Hoofdstad - Bruxelles-Capitale, Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest, 1060, België / Belgique / Belgien',
   'in_house_nbr': '20',
   'lat': '50.8358677',
   'lon': '4.3385087',
   'lpost_house_nbr': '20',
   'lpost_unit': '',
   'method': 'orig',
   'osm_addr_in': 'Av. Fonsny, 20, 1060 Saint-Gilles, Belgium',
   'place_id': 182128,
   'place_rank': 30}],
 'rejected': [{'SIM_street': '',
   'SIM_zip': '',
   'addr_out_city': 'Saint-Gilles - Sint-Gillis',
   'addr_out_country': 'België / Belgique / Belgien',
   'addr_out_number': '20',
   'addr_out_postcode': '1060',
   'addr_out_street': 'Avenue Fonsny - Fonsnylaan',
   'displa

In [272]:
# sum(res["timing"].values())

## Batch calls (row by row)

In [223]:
addresses = get_addresses("address.csv.gz")
# addresses = addresses.sample(10000, replace=True).copy()
addresses

,addr_key,CountryFR,postcode,city,street,housenumber
0,2762939,Belgique,2240,Zandhoven,Kriekenlaan,22
1,2253494,Belgique,8600,Diksmuide,Cardijnlaan(D),*
2,1764140,Belgique,9860,Oosterzele,Poststraat,19
3,87070,Belgique,8300,Knokke-Heist,Sylvain Dupuisstraat,26
4,2261339,Belgique,3630,Maasmechelen,Oude Baan(M),338
...,...,...,...,...,...,...
995,2720764,Belgique,5060,Sambreville,Rue du Wainage(VEL),62
996,386196,Belgique,7520,Tournai,Chaussée de Tournai(R-C),4
997,1474962,Belgique,7331,Saint-Ghislain,Rue Louis Caty(B),32
998,2424889,Belgique,7090,Braine-le-Comte,Drève des Bouleaux,5



### Simple way

In [270]:
t = datetime.now()
addresses["json"] = addresses.progress_apply(call_ws, check_result=False, structured_osm=False, axis=1)
tot_time = (datetime.now() - t).total_seconds()
print(f"{tot_time} seconds, {addresses.shape[0]/tot_time} it/s")
# 5.14, 4.83 it/s
# 2.08 8.92

  0%|          | 0/1000 [00:00<?, ?it/s]

132.750946 seconds, 7.532903004698738 it/s


In [52]:
addresses.iloc[5].json

{'match': [{'SIM_street_which': '',
   'addr_out_city': 'Uccle - Ukkel',
   'addr_out_country': 'België / Belgique / Belgien',
   'addr_out_number': '55',
   'addr_out_postcode': '1180',
   'addr_out_street': 'Bosveldweg',
   'display_name': '55, Bosveldweg, Churchill, Uccle - Ukkel, Brussel-Hoofdstad - Bruxelles-Capitale, Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest, 1180, België / Belgique / Belgien',
   'in_house_nbr': '55',
   'lat': '50.8076561',
   'lon': '4.3605836415694705',
   'lpost_house_nbr': '55',
   'lpost_unit': '',
   'method': 'orig',
   'osm_addr_in': 'Bosveldweg, 55, 1180 Uccle, Belgique',
   'place_id': 959618,
   'place_rank': 30}],
 'rejected': [],
 'timing': {'checker': 0.0,
  'libpostal': 0.0,
  'osm': 0.015169,
  'osm_post': 0.028245,
  'photon': 0.0,
  'transformer': 0.006084},
 'time': 0.090027}

In [53]:
sum(addresses.iloc[5].json["timing"].values())

0.049498

### Using Dask

In [273]:
t = datetime.now()
dd_addresses = dd.from_pandas(addresses, npartitions=8)

dask_task = dd_addresses.apply(call_ws, check_result=False, meta=('x', 'str'), axis=1)

with ProgressBar(): 
    addresses["json"] = dask_task.compute()
    
tot_time = (datetime.now() - t).total_seconds()
print(f"{tot_time} seconds, {addresses.shape[0]/tot_time} it/s")

[########################################] | 100% Completed | 25.5s
25.543355 seconds, 39.14912508556531 it/s


In [ ]:
# 1000, 1 worker: 4m18
# 4 workers, npart=4 : 1m20
# 8 workers, npart=4 : 1m20
# 8 workers, npart=8 : 44s

# with checker=False:
# 8 workers, npart=8 : 24s


In [ ]:
expand_json(addresses)

In [271]:
# addresses.json.loc[550]

## Batch calls (batch WS)

In [28]:
addresses = pd.read_csv(f"../GISAnalytics/data/geocoding/kbo_1000_sample.csv")
addresses = addresses.rename(columns={"Unnamed: 0": addr_key_field, "address": street_field})
addresses[city_field] = ""
addresses[country_field] =  "Belgique"
addresses[housenbr_field] = ""
addresses[postcode_field]=""
addresses

,addr_key,CountryFR,postcode,city,street,housenumber,street
0,2762939,Belgique,,,Kriekenlaan,,"Kriekenlaan, 22, 2240 Zandhoven, Belgique"
1,2253494,Belgique,,,Cardijnlaan(D),,"Cardijnlaan(D), *, 8600 Diksmuide, Belgique"
2,1764140,Belgique,,,Poststraat,,"Poststraat, 19, 9860 Oosterzele, Belgique"
3,87070,Belgique,,,Sylvain Dupuisstraat,,"Sylvain Dupuisstraat, 26, 8300 Knokke-Heist, B..."
4,2261339,Belgique,,,Oude Baan(M),,"Oude Baan(M), 338, 3630 Maasmechelen, Belgique"
...,...,...,...,...,...,...,...
995,2720764,Belgique,,,Rue du Wainage(VEL),,"Rue du Wainage(VEL), 62, 5060 Sambreville, Bel..."
996,386196,Belgique,,,Chaussée de Tournai(R-C),,"Chaussée de Tournai(R-C), 4, 7520 Tournai, Bel..."
997,1474962,Belgique,,,Rue Louis Caty(B),,"Rue Louis Caty(B), 32, 7331 Saint-Ghislain, Be..."
998,2424889,Belgique,,,Drève des Bouleaux,,"Drève des Bouleaux, 5, 7090 Braine-le-Comte, B..."


In [187]:
addresses

,addr_key,CountryFR,postcode,city,street,housenumber,osm,address,json
845,2160439,Belgique,4540,Amay,Avenue de Dieuze(AMP),10,[],"Avenue de Dieuze(AMP), 10, 4540 Amay Belgique","{'match': [{'SIM_street_which': '', 'addr_out_..."
951,3030707,Belgique,6200,Châtelet,Rue de Gilly,3,"[{'place_id': 3066672, 'lat': '50.40920605', '...","Rue de Gilly, 3, 6200 Châtelet Belgique","{'match': [{'SIM_street_which': '', 'addr_out_..."
298,717934,Belgique,3740,Bilzen,Zangerheistraat,31,"[{'place_id': 3573074, 'lat': '50.88994245', '...","Zangerheistraat, 31, 3740 Bilzen Belgique","{'match': [{'SIM_street_which': '', 'addr_out_..."
680,1796283,Belgique,3600,Genk,Oudasserstraat,33,"[{'place_id': 440251, 'lat': '50.98279665', 'l...","Oudasserstraat, 33, 3600 Genk Belgique","{'match': [{'SIM_street_which': '', 'addr_out_..."
611,2282957,Belgique,1800,Vilvoorde,Mechelsesteenweg,2A,"[{'place_id': 1294956, 'lat': '50.9339683', 'l...","Mechelsesteenweg, 2A, 1800 Vilvoorde Belgique","{'match': [{'SIM_street_which': '', 'addr_out_..."
...,...,...,...,...,...,...,...,...,...
323,2216417,Belgique,1790,Affligem,Tenbos,3,"[{'place_id': 3491627, 'lat': '50.9058866', 'l...","Tenbos, 3, 1790 Affligem Belgique","{'match': [{'SIM_street_which': '', 'addr_out_..."
315,2753787,Belgique,7711,Mouscron,Rue du Trieu(D),144,"[{'place_id': 403137, 'lat': '50.7237255', 'lo...","Rue du Trieu(D), 144, 7711 Mouscron Belgique","{'match': [{'SIM_street_which': '', 'addr_out_..."
991,1741686,Belgique,2870,Puurs-Sint-Amands,Lelielaan,6,"[{'place_id': 1903722, 'lat': '51.0473536', 'l...","Lelielaan, 6, 2870 Puurs-Sint-Amands Belgique","{'match': [{'SIM_street_which': '', 'addr_out_..."
186,430222,Belgique,9230,Wetteren,Brusselsesteenweg,415,"[{'place_id': 1868840, 'lat': '50.9918723', 'l...","Brusselsesteenweg, 415, 9230 Wetteren Belgique","{'match': [{'SIM_street_which': '', 'addr_out_..."


### Single block

In [274]:
# Only geocoding
# addresses["StreetFR"] = ""
t = datetime.now()

call_ws_batch(addresses[[addr_key_field, street_field, housenbr_field, postcode_field, city_field, country_field]], mode="geo", check_result=False, structured_osm=True)

tot_time = (datetime.now() - t).total_seconds()
print(f"{tot_time} seconds, {addresses.shape[0]/tot_time} it/s")

24.384449 seconds, 41.009743545978836 it/s


In [249]:
# Geocode + address
call_ws_batch(addresses[[addr_key_field, street_field, housenbr_field, postcode_field, city_field, country_field]], mode="short", check_result=False) 

,addr_key,lat,lon,place_rank,method,place_id,addr_out_street,addr_out_number,in_house_nbr,lpost_house_nbr,lpost_unit,addr_out_postcode,addr_out_city,addr_out_country
0,2762939,51.211615,4.649820,30.0,orig,1032308.0,Kriekenlaan,22,22,22,,2240,Zandhoven,België / Belgique / Belgien
1,2253494,51.059548,2.870803,21.0,nostreet,4005124.0,None,None,*,,,8600,Diksmuide,België / Belgique / Belgien
2,1764140,50.924678,3.788056,26.0,orig,436186.0,Poststraat,None,19,19,,9860,Issegem,België / Belgique / Belgien
3,87070,51.351333,3.285724,30.0,orig,3675084.0,Sylvain Dupuisstraat,24;26;28,26,26,,8300,Knokke-Heist,België / Belgique / Belgien
4,2261339,50.977155,5.690444,26.0,orig,379663.0,Oude Baan,None,338,,m 338,3630,Maasmechelen,België / Belgique / Belgien
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2720764,50.462361,4.596458,30.0,regex[init],2657266.0,Rue du Wainage,62,62,62,,5060,Sambreville,België / Belgique / Belgien
996,386196,50.639445,3.349058,26.0,orig,1983533.0,Chaussée de Tournai,None,4,4,,7520,Ramegnies-Chin,België / Belgique / Belgien
997,1474962,50.479737,3.838976,26.0,orig,432987.0,Rue Louis Caty,None,32,32,,7331,Saint-Ghislain,België / Belgique / Belgien
998,2424889,50.606398,4.159495,26.0,orig,379561.0,Drève des Bouleaux,None,5,5,,7090,Braine-le-Comte,België / Belgique / Belgien


In [ ]:
# Geocode + address, with rejected addresses
call_ws_batch(addresses, mode="long", with_reject=True)

In [ ]:
# call_ws_batch(addresses[addresses.EntityNumber.str.startswith("0554.81")], mode="long", with_reject=True)

In [ ]:
# a[a.in_house_nbr.str.upper() != a.lpost_house_nbr.str.upper()]

### Batch blocs

In [275]:
def call_ws_batch_chunks(addr_data, mode="geo", with_reject=False, check_result=True, structured_osm=False, chunk_size=100): 
    ## TODO : find a better way with dask? It seems that map_partitions does not support function returning dataframes. 

    chunks = np.array_split(addr_data, addr_data.shape[0]//chunk_size)

    res= [call_ws_batch(chunk, mode=mode, 
                        check_result=check_result, 
                        structured_osm=structured_osm) for chunk in tqdm(chunks)]
    df_res = pd.concat(res, sort=False)
    return df_res

In [277]:
t = datetime.now()

df_res = call_ws_batch_chunks(addresses, chunk_size=100, mode="short", check_result=False)

tot_time = (datetime.now() - t).total_seconds()
print(f"{tot_time} seconds, {addresses.shape[0]/tot_time} it/s")

df_res

  0%|          | 0/10 [00:00<?, ?it/s]

38.862636 seconds, 25.731656493913587 it/s


,addr_key,lat,lon,place_rank,method,place_id,addr_out_street,addr_out_number,in_house_nbr,lpost_house_nbr,lpost_unit,addr_out_postcode,addr_out_city,addr_out_country
0,2762939,51.211615,4.649820,30.0,orig,1032308.0,Kriekenlaan,22,22,22,,2240,Zandhoven,België / Belgique / Belgien
1,2253494,51.059548,2.870803,21.0,nostreet,4005124.0,None,None,*,,,8600,Diksmuide,België / Belgique / Belgien
2,1764140,50.924678,3.788056,26.0,orig,436186.0,Poststraat,None,19,19,,9860,Issegem,België / Belgique / Belgien
3,87070,51.351333,3.285724,30.0,orig,3675084.0,Sylvain Dupuisstraat,24;26;28,26,26,,8300,Knokke-Heist,België / Belgique / Belgien
4,2261339,50.977155,5.690444,26.0,orig,379663.0,Oude Baan,None,338,,m 338,3630,Maasmechelen,België / Belgique / Belgien
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2720764,50.462361,4.596458,30.0,regex[init],2657266.0,Rue du Wainage,62,62,62,,5060,Sambreville,België / Belgique / Belgien
96,386196,50.639445,3.349058,26.0,orig,1983533.0,Chaussée de Tournai,None,4,4,,7520,Ramegnies-Chin,België / Belgique / Belgien
97,1474962,50.479737,3.838976,26.0,orig,432987.0,Rue Louis Caty,None,32,32,,7331,Saint-Ghislain,België / Belgique / Belgien
98,2424889,50.606398,4.159495,26.0,orig,379561.0,Drève des Bouleaux,None,5,5,,7090,Braine-le-Comte,België / Belgique / Belgien


In [300]:
# import dask.array as da

# t = datetime.now()

# chunks = np.array_split(addresses,addresses.shape[0]//100)

# dd_addresses = da.from_array(chunks, chunks= (10, 100, 7))

# dask_task = dd_addresses.apply(lambda a: print(display(a)) ,  meta=('x', 'str'), axis=1)

# with ProgressBar(): 
#     addresses["json"] = dask_task.compute()
    
# tot_time = (datetime.now() - t).total_seconds()
# print(f"{tot_time} seconds, {addresses.shape[0]/tot_time} it/s")

In [301]:
# chunks[5]

In [299]:
# dd_addresses = dd.from_pandas(addresses, npartitions=8)

# dask_task = dd_addresses.apply(call_ws, check_result=False, meta=('x', 'str'), axis=1)

# with ProgressBar(): 
#     addresses["json"] = dask_task.compute()
    
# tot_time = (datetime.now() - t).total_seconds()
# print(f"{tot_time} seconds, {addresses.shape[0]/tot_time} it/s")

In [ ]:
# df_res[df_res.addr_out_number.str.upper() != df_res.lpost_house_nbr.str.upper()]

In [ ]:
df_res.method.value_counts()

## Comparing options

In [ ]:
addresses = get_addresses("address.csv.gz")
addresses = addresses[addresses[country_field] == "Belgique"]
addresses = addresses.sample(10000).copy()

In [ ]:
results = {}
it_per_seconds=pd.DataFrame()

for check_label in ["check", "nocheck"]:
    for struct_label in ["struct", "unstruct" ]:
        print(check_label, struct_label)
        start=datetime.now()
        
        results[(check_label, struct_label)] = call_ws_batch_chunks(addresses, 
                                                                    mode="short", 
                                                                    check_result   =  check_label == "check", 
                                                                    structured_osm =  struct_label == "struct")
        
        it_per_seconds.loc[check_label, struct_label] = addresses.shape[0] / (datetime.now()-start).total_seconds()
print("Iterations per seconds:")
it_per_seconds

In [ ]:
print("Match rate")
pd.DataFrame({k1: {k2: results[(k1,k2)].shape[0]/addresses.shape[0] for k2 in ["struct", "unstruct"]} 
                  for k1 in  ["check","nocheck"]})

In [ ]:
print("Match rate (without nostreet)")
pd.DataFrame({k1: {k2: results[(k1,k2)].query("method!='nostreet'").shape[0]/addresses.shape[0] for k2 in ["struct", "unstruct"]} 
                  for k1 in  ["check","nocheck"]})

In [ ]:
print("Unmatched addresses")
for k1 in results:
    print(k1)
    nomatch=addresses[~addresses[addr_key_field].isin(results[k1]["addr_key"])]
    display(nomatch)
    print(nomatch[country_field].value_counts())

In [ ]:
vc_values = pd.DataFrame(columns=results.keys(), index=results.keys())

for k1 in results:
    vc_values.loc[k1, k1] = results[k1].shape[0]
    for k2 in results:
        if k1>k2:
            r1=results[k1]
            r2=results[k2]
            mg = r1[["addr_key", "place_id"]].merge(r2[["addr_key", "place_id"]], on="addr_key", how="outer", indicator=True)
 
            vc = mg._merge.value_counts()

            mismatches = mg[mg.place_id_x != mg.place_id_y][["addr_key"]]
            mismatches = mismatches.merge(addresses.rename({addr_key_field:"addr_key"}, axis=1))
            mismatches = mismatches.merge(r1[["addr_key", "addr_out_street", "addr_out_number", "extra_house_nbr", "addr_out_postcode", "addr_out_city"]], on="addr_key")
            mismatches = mismatches.merge(r2[["addr_key", "addr_out_street", "addr_out_number", "extra_house_nbr", "addr_out_postcode", "addr_out_city"]], on="addr_key")
            mismatches.columns = pd.MultiIndex.from_arrays([["Input"]*6 + [f"x:{k1}"]*5 + [f"y:{k2}"]*5, mismatches.columns])

            mismatch_values = mismatches[(mismatches[f"x:{k1}"].rename(lambda x: x.replace("_x", ""), axis=1).fillna("") != 
                                          mismatches[f"y:{k2}"].rename(lambda x: x.replace("_y", ""), axis=1).fillna("")).any(axis=1)]
            
            mismatch_values_no_nmbr = mismatches[(mismatches[f"x:{k1}"].rename(lambda x: x.replace("_x", ""), axis=1).drop("addr_out_number", axis=1).fillna("") != 
                                                  mismatches[f"y:{k2}"].rename(lambda x: x.replace("_y", ""), axis=1).drop("addr_out_number", axis=1).fillna("")).any(axis=1)]
            
            
            vc_label = f"{vc['both']} ({mismatches.shape[0]} - {mismatch_values.shape[0]} - {mismatch_values_no_nmbr.shape[0]}) / {vc['left_only']} / {vc['right_only']}"
            vc_values.loc[k1, k2]=vc_label

                
            print(f"{k1} vs {k2}")
            print(vc_label)
            print("-----------------------------")
            
            print(f"Only in {k1}")
            display(r1[r1.addr_key.isin(mg[mg._merge=="left_only"].addr_key)].merge(addresses.rename({addr_key_field:"addr_key"}, axis=1)))
            
            print(f"Only in {k2}")
            display(r2[r2.addr_key.isin(mg[mg._merge=="right_only"].addr_key)].merge(addresses.rename({addr_key_field:"addr_key"}, axis=1)))
            
            print("Mismatch on place_id")
            display(mismatches)
            
            print("Mismatch on values")
            
            display(mismatch_values)
            
            print("Mismatch on values (no nbr)")
            display(mismatch_values_no_nmbr)
            
            print("#######################")
            
# display(vc_values.fillna(""))

In [ ]:
print("Common in both (disagree on place_id - disagree on values - disagree on values, ignoring number) / results only for row / results only for columns")
vc_values.fillna("")

# tests

In [34]:
osm_host ="172.27.0.64:8080"
def get_osm(addr, accept_language = ""): #lg = "en,fr,nl"
    params = urllib.parse.urlencode({"q": addr,
                                    "format":"jsonv2",
                                    "accept-language":accept_language,
                                    "addressdetails":"1",
                                    "namedetails" : "1",
                                    "limit": "50"
                                    })
    
    url = "http://%s/search.php?%s"%(osm_host, params)
#     print(f"Call to OSM: {url}")
    try: 
        with urllib.request.urlopen(url) as response:
            res = response.read()
            res = json.loads(res)
#             return res
            return [ {field: item[field] for field in ["place_id", "lat", "lon", "display_name", "address", "namedetails", "place_rank", "category", "type"]} for item in res] 
    except Exception as e:
        raise Exception (f"Cannot get OSM results ({osm_host}): {e}") 

In [55]:
%timeit get_osm("Chaussée de Tervueren 59, 1160 Auderghem")

8.57 ms ± 66 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [87]:
%timeit get_osm("Av. Fonsny 20, 1060 Bruxelles")

9.81 ms ± 118 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [49]:
addresses["osm"] = addresses.address.progress_apply(get_osm)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [45]:
addresses["address"] = addresses[street_field]+", "+addresses[housenbr_field].fillna("")+", " \
                            +addresses[postcode_field]+" "+addresses[city_field]+" "+addresses[country_field]

In [47]:
# addresses[addresses.address.isnull()]

In [124]:
%timeit call_ws_test({street_field:   "Av. Fonsny", \
         housenbr_field: "20",\
         city_field:     "Saint-Gilles",\
         postcode_field: "1060",\
         country_field:  "Belgium"}, check_result=False, structured_osm=False)
# res

20.5 ms ± 165 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [122]:
call_ws_test({street_field:   "Av. Fonsny", \
         housenbr_field: "20",\
         city_field:     "Saint-Gilles",\
         postcode_field: "1060",\
         country_field:  "Belgium"}, check_result=False, structured_osm=False)
# res

{'res': [{'address': {'country': 'Belgique',
    'country_code': 'be',
    'county': 'Bruxelles-Capitale',
    'house_number': '20',
    'postcode': '1060',
    'region': 'Bruxelles-Capitale',
    'road': 'Avenue Fonsny',
    'town': 'Saint-Gilles'},
   'category': 'place',
   'display_name': '20, Avenue Fonsny, Saint-Gilles, Bruxelles-Capitale, 1060, Belgique',
   'lat': '50.8358677',
   'lon': '4.3385087',
   'namedetails': {},
   'place_id': 182128,
   'place_rank': 30,
   'type': 'house'},
  {'address': {'country': 'Belgique',
    'country_code': 'be',
    'county': 'Bruxelles-Capitale',
    'house_number': '20',
    'place': 'DAE (Smals)',
    'postcode': '1060',
    'region': 'Bruxelles-Capitale',
    'road': 'Avenue Fonsny',
    'town': 'Saint-Gilles'},
   'category': 'place',
   'display_name': 'DAE (Smals), 20, Avenue Fonsny, Saint-Gilles, Bruxelles-Capitale, 1060, Belgique',
   'lat': '50.8358216',
   'lon': '4.3386884',
   'namedetails': {'name': 'DAE (Smals)'},
   'place_id

In [257]:
# %timeit 
res =call_ws({street_field:   "Av. Fonsny", \
         housenbr_field: "20",\
         city_field:     "Saint-Gilles",\
         postcode_field: "1060",\
         country_field:  "Belgium"}, check_result=False, structured_osm=False)
res

{'match': [{'SIM_street_which': '',
   'addr_out_city': 'Saint-Gilles - Sint-Gillis',
   'addr_out_country': 'België / Belgique / Belgien',
   'addr_out_number': '20',
   'addr_out_postcode': '1060',
   'addr_out_street': 'Avenue Fonsny - Fonsnylaan',
   'display_name': '20, Avenue Fonsny - Fonsnylaan, Saint-Gilles - Sint-Gillis, Brussel-Hoofdstad - Bruxelles-Capitale, Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest, 1060, België / Belgique / Belgien',
   'in_house_nbr': '20',
   'lat': '50.8358677',
   'lon': '4.3385087',
   'lpost_house_nbr': '20',
   'lpost_unit': '',
   'method': 'orig',
   'osm_addr_in': 'Av. Fonsny, 20, 1060 Saint-Gilles, Belgium',
   'place_id': 182128,
   'place_rank': 30}],
 'rejected': [{'SIM_street': '',
   'SIM_zip': '',
   'addr_out_city': 'Saint-Gilles - Sint-Gillis',
   'addr_out_country': 'België / Belgique / Belgien',
   'addr_out_number': '20',
   'addr_out_postcode': '1060',
   'addr_out_street': 'Avenue Fonsny - Fonsnylaan',
   'displa

In [214]:
call_ws_test({street_field:   "Av. Fonsny", \
         housenbr_field: "20",\
         city_field:     "Saint-Gilles",\
         postcode_field: "1060",\
         country_field:  "Belgium"})

{'key': 4537490478458538424, 'time': 0.007811}

In [215]:
addresses.progress_apply(call_ws_test, check_result=False, structured_osm=False, axis=1)

  0%|          | 0/1000 [00:00<?, ?it/s]

845    {'key': -7446963526649159374, 'time': 0.011174}
951       {'key': 762473121338280903, 'time': 0.00671}
298     {'key': 9144190004510795113, 'time': 0.006501}
680    {'key': -4959491702206956535, 'time': 0.006234}
611    {'key': -1344177550255083595, 'time': 0.006133}
                            ...                       
323    {'key': -8613962303765128288, 'time': 0.006017}
315    {'key': -8327614734770250404, 'time': 0.005897}
991     {'key': 7750053913333578829, 'time': 0.005916}
186    {'key': -5417820216234844161, 'time': 0.005949}
56      {'key': 7125128659801556648, 'time': 0.006021}
Length: 1000, dtype: object

In [172]:
tm =res["timing"] 
sum(res["timing"].values()) - res["timing"]["global"]

141.40799999999996

In [173]:
tm["init_df"] + tm["t&p"] + tm["extra_hn"] + tm["format_res"]

84.55499999999999